### Detect fiducial particles in EM image

In [1]:
%matplotlib inline

import cv2
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from imutils.object_detection import non_max_suppression
from utils import list_to_dataframe, dataframe_to_nparray, dataframe_to_xml, dataframe_to_pointcloud


def plot_image(image):    
    """
    Plot a grayscale image in the original size - takes time
    """
    fig = plt.figure(figsize=(40, 40))
    ax1 = plt.subplot(1, 1, 1) 
    ax1.imshow(image, cmap='gray')  


In [ ]:
from pathlib import Path

# Load the main image and template
input_folder = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1')
input_file = '240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_small.tif'
image_path = input_folder / input_file

output_folder = input_folder / 'output'
output_folder.mkdir(exist_ok=True)

#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_small.tif')
#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos2/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos2_bin4_EM.tif')
#image_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos3/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos3_bin4_EM.tif')

print(image_path.exists())

#template_path = Path('E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1/240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM_template.tif')
#print(template_path.exists())

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)

# Parameters
matching_threshold = 0.7  # threshold for template matching
overlap_threshold =0.1

# Get template dimensions
#h, w = template.shape

# Information about fiducial particles
fiducial_diam = 30     # diameter of fiducial particles in px

#### Creating the template

Fiducial particles have a very distinct center, characterized by a dark dot surrounded by a gray region. Instead of attempting to match the entire particle using a full template, we focus on detecting the central region only. To accomplish this, the template can be easily generated artificially.

In [3]:
# Create an empty template of size ('size','size') filled with constant value - 'value'
size = 9  
value = 80
template = np.full((size, size), value, dtype=np.uint8)

# Define the 3x3 pattern
pattern = np.array([[4, 2, 4],
                    [2, 0, 2],
                    [4, 2, 4]])

# Calculate the starting index to place the pattern in the middle of the template
s_idx = (template.shape[0] - pattern.shape[0]) // 2
e_idx = s_idx + pattern.shape[0]
    
# Place the pattern in the middle of the template
template[s_idx:e_idx, s_idx:e_idx] = pattern

#import bigfish.stack as stack
#template = stack.resize_image(template,(template.shape[0]*2,template.shape[0]*2), method='bilinear')

Sample 1 bin 2 is has higher resolution so it seams - so there is larger template needed:

In [4]:
# Create an empty template of size ('size','size') filled with constant value - 'value'
size = 11  
value = 80
template = np.full((size, size), value, dtype=np.uint8)

# Define the 5x5 pattern
pattern = np.array([[32, 8, 4, 8, 32],
                    [8, 4, 2, 4, 8],
                    [4, 2, 0, 2, 4],
                    [8, 4, 2, 4, 8],
                    [32, 8, 4, 8, 32]])

# Calculate the starting index to place the pattern in the middle of the template
s_idx = (template.shape[0] - pattern.shape[0]) // 2
e_idx = s_idx + pattern.shape[0]
    
# Place the pattern in the middle of the template
template[s_idx:e_idx, s_idx:e_idx] = pattern

In [ ]:
plot_image(template)
print(template)

In [33]:
# Display the input image
# plot_image(image)

In [5]:
def non_max_suppression(boxes, scores, threshold):
    # Sort boxes by score in descending order
    sorted_indices = np.argsort(scores)[::-1]
    print("Indices: ", sorted_indices)
    
    keep_boxes = []
    
    while sorted_indices.size > 0:
        # Pick the box with the highest score
        box_id = sorted_indices[0]
        keep_boxes.append(box_id)
        
        # Calculate IoU of the picked box with the rest
        ious = calculate_iou(boxes[box_id], boxes[sorted_indices[1:]])
        
        # Remove boxes with IoU over the threshold
        keep_indices = np.where(ious < threshold)[0]
        
        # Update the indices
        sorted_indices = sorted_indices[keep_indices + 1]
    print("Keep boxes: ", keep_boxes)
    return keep_boxes

def calculate_iou(box, boxes):
    # Calculate intersection areas
    x1 = np.maximum(box[0], boxes[:, 0])
    y1 = np.maximum(box[1], boxes[:, 1])
    x2 = np.minimum(box[2], boxes[:, 2])
    y2 = np.minimum(box[3], boxes[:, 3])
    
    intersection_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    
    # Calculate union areas
    box_area = (box[2] - box[0]) * (box[3] - box[1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])
    union_area = box_area + boxes_area - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    return iou

def template_matching(image, template, threshold):
    result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    #plot_image(result)
    locations = np.where(result >= threshold)
    scores = result[locations]
    matches = list(zip(*locations[::-1]))
    
    return matches, scores

def average_perimeter_intensity(image, center, radius):
    # Create a circular mask
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    cv2.circle(mask, center, radius, 255, 1)
    
    # Extract perimeter pixels
    perimeter_pixels = image[mask == 255]
    
    # Calculate average intensity
    average_intensity = np.mean(perimeter_pixels)
    
    return average_intensity

In [ ]:
# Perform template matching
matches,scores = template_matching(image, template, matching_threshold)

#print("Scores:", scores)
#print("Match locations: ", matches)


# Create bounding boxes
w, h = template.shape[::-1]
boxes = [(x, y, x + w, y + h) for (x, y) in matches]
#print("Boxes: ", boxes)

# Apply non-maximum suppression for filtering out overlapping boxes
keep_ids = non_max_suppression(np.array(boxes), scores, overlap_threshold)

#print("Kept_boxes: ", np.array(keep_ids))
#print(matches[keep_ids[0]])

# Do not filter out the matches
#loc = [matches[keep_id] for keep_id in keep_ids]

# Filter out the matches with average perimeter intensity over 110
loc = [matches[keep_id] for keep_id in keep_ids if average_perimeter_intensity(image, matches[keep_id], 6) < 110]


# Draw rectangles around the matched regions
img2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
for pt in loc:
    cv2.rectangle(img2, pt, (pt[0] + w, pt[1] + h), (0, 255, 0), 2)
    

In [ ]:
# Save FP position saved in loc variable in a Pandas dataframe with columns: 'id', 'name', 'pos_x', 'pos_y'

target_df = list_to_dataframe(loc) #str(output_folder/"target_df.csv")
print(target_df)

# Convert dataframe into numpy array of coordinate pairs (X,Y), XML file and PLY file
#target = dataframe_to_nparray(target_df)
#dataframe_to_xml(target_df)       # str(output_folder/"target.xml")
target_pcd = dataframe_to_pointcloud(target_df, str(output_folder/"target-test.ply"))  # "str(output_folder/target.ply)"

In [ ]:
from matplotlib import pyplot as plt
plot_image(img2)
plt.savefig('fiducial_detection.png')

### Filtering of the detected fiducial particles 

Clusters of fiducial particles (FP) will be replaced by 1 point located in the centre of the cluster. Single FP will be removed.

In [ ]:
# Create a black image of the same size as img2, then draw filled circles with the radius of FP at the locations 
# of FP taken from the loc variable, dilate the binary image by circular kernel to connect nearby FP, save the image

img_mask = np.zeros_like(img2)
for pt in loc:
    # draw a filled circle around the fiducial particle
    cv2.circle(img_mask, (pt[0] + w//2, pt[1] + h//2), fiducial_diam//2, (255, 255, 255), -1)
    # draw a filled rectangle around the fiducial particle
    #cv2.rectangle(img_mask, pt, (pt[0] + w, pt[1] + h), (255, 255, 255), -1)

# Dilate img_mask by circluar kernel of size 7x7
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
img_mask = cv2.dilate(img_mask, kernel, iterations=1)

plot_image(img_mask)
plt.savefig('fiducial_detection_mask.png')


In [ ]:
# Calculate for each connected component the centroid and the pixel area, if the area is larger then 
# the area of 3 fiducial particles then keep the FP, save the centroid and the area into a list

# Find connected components
_, labels = cv2.connectedComponents(img_mask)

# Calculate the centroid and area of each connected component
centroids = []
areas = []

for label in np.unique(labels):
    if label == 0:             # skip the background
        continue
    
    mask = np.zeros_like(img_mask, dtype=np.uint8)
    mask[labels == label] = 1
    
    moments = cv2.moments(mask)
    print("Moments: ", moments)

    if moments["m00"] > 1500:   # 2 * fiducial_diam * fiducial_diam:
        centroids.append((int(moments["m10"] / moments["m00"]), int(moments["m01"] / moments["m00"])))
        areas.append(int(moments["m00"]))
        #print(moment["m00"])

print("Centroids: ", centroids)
print("Areas: ", areas)

# Draw the centroids on the original image
img3 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
for centroid in centroids:
    cv2.circle(img3, centroid, 5, (0, 255, 0), -1)

plot_image(img3)
plt.savefig('fiducial_detection_centroids.png')

# Save the centroids and areas into a text file
output_path = Path('filtered_fiducial_detection.txt')
with open(output_path, 'w') as f:
    for centroid, area in zip(centroids, areas):
        f.write(f"{centroid[0]},{centroid[1]},{area}\n")

print("Output saved to: ", output_path)




Save the FP locations and filtered FP locations into a pandas dataframe and then point cloud

In [ ]:
# Save FP position saved in centroids variable into a Pandas dataframe with columns: 'id', 'name', 'pos_x', 'pos_y'

target_s_df = list_to_dataframe(centroids) #str(output_folder/"target_s_df.csv")
print(target_s_df)

# Convert dataframe into numpy array of coordinate pairs (X,Y), XML file and PLY file
#target_s = dataframe_to_nparray(target_s_df)
#dataframe_to_xml(target_s_df)       # str(output_folder/"target.xml")
target_s_pcd = dataframe_to_pointcloud(target_s_df, str(output_folder/"target-s-test.ply"))  # "target.ply"
